# Asset Pricing and Portfolio Management #

Suppose that an investor owns, on September 25th, 2008 a portfolio worth \$ 10 million consisting of
investments in four stock indices: Dow Jones Industrial Average (DJIA) in the United States, the FTSE
100 in the United Kingdom, the CAC 40 in France, and the Nikkei 225 in Japan. So, let’s suppose that today is September 25th, 2008.
The value of the investment in each index on September 25, 2008, is (in \$ 000s): \$ 4000 in DJIA, \$ 3000
in FTSE, \$ 1000 in CAC and \$ 2000 in NIKKEI.

## Question 1 : Using a GARCH model (1,1), estimate the tomorrow's volatility of each of the four indices. Compare the values obtained. Is the result in line with your expectations ? ##



The $garch(p,q)$ model calculates the daily volatility forecast from the $p$ most recent observations $u$
and the $q$ most recent variances $\sigma^2$ :

$$ \sigma_t^2 = \omega + \sum_{i=0}^{p}\alpha_i u_{t-i}^2 + \sum_{i=0}^{q}\beta_i \sigma_{t-i}^2$$

$$ \sigma_t^2 = \omega + \alpha_0 u_t^2 + \alpha_1 u_{t-1}^2 + ... + \alpha_p u_{t-p}^2 + \beta_0 \sigma_t^2 + \beta_1 \sigma_{t-1}^2 + ... + \beta_q \sigma_{t-q}^2$$

With the simplest and most popular $garch(1,1)$ model : \
$$ \sigma_t^2 = \omega + \alpha u_{t-1}^2 + \beta \sigma_{t-1}^2 $$
with :
* $u_{t-1}$ the market latest news
* $\sigma_{t-1}$ the lastest standard deviation

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from arch.univariate import arch_model
import datetime

In [15]:
indexes    = pd.read_excel('VaR.xls','for_python',index_col=0)
portfolio  = [4000,3000,1000,2000]
indexes

,DJIA,FTSE-500,CAC-40,Nikkei
Day,,,,
0,11219.38,11131.84224,6373.894033,131.774435
1,11173.59,11096.28032,6378.161510,134.381821
2,11076.18,11185.35030,6474.040196,135.943301
3,11124.37,11016.70812,6357.485948,135.438090
4,11088.02,11040.72970,6364.764458,134.100284
...,...,...,...,...
496,11019.69,8878.18400,5689.850489,109.547101
497,11388.44,9734.01951,6230.005762,111.618539
498,11015.69,9656.26083,6181.952576,113.228975


In [38]:
# We start with the calculation of returns (we have the index LEVELS and not the returns)
returns = pd.DataFrame()
for column_name in indexes.columns:
    returns.insert(len(returns.columns), column_name , indexes[column_name].pct_change() + 1)
returns = returns.iloc[1: , :] ###WHY?

# We use normalized returns for the GARCH model to focus on the variance
normalized_returns = returns - np.mean(returns)

In [39]:
# p=1, q=1 : forecast lengths. Both at 1 for garch(1, 1)
# o=0 : no integration (different from IGARCH)
one_day_variances = []

for column_name in indexes.columns:
    am = arch_model(100 * np.array(normalized_returns[column_name]), mean='zero',p=1, o=0, q=1)
    garch_result = am.fit()
    print(garch_result.summary())
    one_day_variances.append(garch_result.params[0] / (1 - garch_result.params[1] - garch_result.params[2]))
    

Iteration:      1,   Func. Count:      5,   Neg. LLF: 267847.3261533039
Iteration:      2,   Func. Count:     11,   Neg. LLF: 701.4429297753443
Iteration:      3,   Func. Count:     16,   Neg. LLF: 3716.666322349747
Iteration:      4,   Func. Count:     21,   Neg. LLF: 817.1578226289529
Iteration:      5,   Func. Count:     26,   Neg. LLF: 699.8670551411744
Iteration:      6,   Func. Count:     31,   Neg. LLF: 1246.419345225507
Iteration:      7,   Func. Count:     36,   Neg. LLF: 269254.04751957895
Iteration:      8,   Func. Count:     43,   Neg. LLF: 701.1986773172243
Iteration:      9,   Func. Count:     48,   Neg. LLF: 833.3076500952891
Iteration:     10,   Func. Count:     53,   Neg. LLF: 692.0031128130515
Iteration:     11,   Func. Count:     58,   Neg. LLF: 689.6821553463033
Iteration:     12,   Func. Count:     62,   Neg. LLF: 689.6168276769419
Iteration:     13,   Func. Count:     76,   Neg. LLF: 2392.147690329456
Iteration:     14,   Func. Count:     86,   Neg. LLF: 723.33891

We check in the summaries that, for each index, the P values of the $\alpha$ and $\beta$ parameters are acceptable, ie that for each $(P>|z|) < 0.05$ : it is not always the case here.

In [41]:
one_day_volatility = np.sqrt(one_day_variances)
one_day_volatility

/var/folders/0y/7yv8khv10d7fyvzwr9l2tbdh0000gn/T/ipykernel_67418/1458409305.py:1: RuntimeWarning: invalid value encountered in sqrt
  one_day_volatility = np.sqrt(one_day_variances)


array([           nan, 5.44635659e+05, 3.98303044e+00, 1.62170553e+00])

[-26136.741291894545,
 296628000608.3771,
 15.864531523862897,
 2.6299288228343762]

In [ ]:
print(res.params)
#Long term volatility $V_L$
VL = res.params[0] / (1 - res.params[1] - res.params[2])
print(np.sqrt(VL))